<a href="https://colab.research.google.com/github/akashwaf/finvasia01/blob/main/simple_opt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# run to install library temporarily 

!git clone https://github.com/Shoonya-Dev/ShoonyaApi-py.git

# this are the libraries mentioned in requirements.txt
!pip install requests
!pip install websocket_client
!pip install /content/ShoonyaApi-py/dist/NorenRestApiPy-0.0.20-py2.py3-none-any.whl
!pip install pyyaml


!pip install pyotp


# !pip install talib-binary             # recently stopped working 
# !pip install TA-Lib-binary               # working now 

Cloning into 'ShoonyaApi-py'...
remote: Enumerating objects: 287, done.
remote: Counting objects: 100% (287/287), done.
remote: Compressing objects: 100% (132/132), done.
remote: Total 287 (delta 153), reused 283 (delta 151), pack-reused 0
Receiving objects: 100% (287/287), 275.64 KiB | 17.23 MiB/s, done.
Resolving deltas: 100% (153/153), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 55 kB 916 kB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing ./ShoonyaApi-py/dist/NorenRestApiPy-0.0.20-py2.py3-none-any.whl
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:

import multiprocessing as mp 
import time
import math
import os , sys
import logging
import pandas as pd 
import pprint
import time
import yaml
import warnings 

# manually installed imports 
import pyotp


import os, time
os.environ['TZ'] = "Asia/Calcutta" # set new timezone
time.tzset()

In [3]:
#### parameters that can be changed 

initial_premium =  25 
new_position_multiplier = 2
sl_multiplier = 1.05
max_sl_multiplier =  3


premium = initial_premium 

sl = premium* sl_multiplier
max_sl = premium* max_sl_multiplier 

expiry = '15-DEC-2022'
selected_instrument = 'BANKNIFTY'





sold_ce = 1
sold_pe = 2

sold_ltp =  {
    
    sold_ce: 0000,
    sold_pe: 0000
  
}






# making global variable and using it in other functions
token_df = pd.read_csv('https://shoonya.finvasia.com/NFO_symbols.txt.zip')

token_df  = token_df [token_df['Symbol'] == selected_instrument]  
token_df  = token_df [token_df['Expiry'] == expiry]
token_df.sort_values(by =  'StrikePrice', inplace = True)
global_token_df = token_df 


ce_token_df  = token_df [token_df['OptionType'] == 'CE']
ce_token_list  = ce_token_df[ 'Token'].tolist()
ce_token_str_list = [str(x) for x in ce_token_list]


pe_token_df  = token_df [token_df['OptionType'] == 'PE']
pe_token_list  = pe_token_df['Token'].tolist()
pe_token_str_list = [str(x) for x in pe_token_list]



ce_lp_data = {}
pe_lp_data = {}
lp_data ={}

feed_opened = False




if True :
    if 'feed_jason' not in globals():    
    feed_opened = False


In [4]:

if True: 
  # warnings.simplefilter("ignore")
  feed_opened = False
  feedJson = {}
  socket_opened = False
  orderJson = {}


  acknowledgement_data = {}
  lp_data = {}
  ce_lp_data = {}
  pe_lp_data = {}



  def event_handler_feed_update(tick_data):
      
      global data_response_type
      global ce_lp_data
      global pe_lp_data
      global lp_data 
      global sold_ltp





      # UPDATE LIVE LTP IN  GLOBAL JASON FORMAT VARIABLE 
      if 'lp' in tick_data:
        lp_data[tick_data['tk']] =   tick_data['lp']


        if tick_data['tk'] in ce_token_str_list :
          ce_lp_data [tick_data['tk']] = tick_data['lp']
        
        
        if tick_data['tk'] in pe_token_str_list :
          pe_lp_data [tick_data['tk']] = tick_data['lp']

             
        
        if tick_data['tk'] == sold_ce or tick_data['tk'] == sold_pe : 

          if tick_data['tk'] == sold_ce: 
            sold_ltp[sold_ce]  =  tick_data['lp']

            
          if tick_data['tk'] == sold_pe: 
            sold_ltp[sold_pe]  =  tick_data['lp']



  def event_handler_order_update(order_data):
      print('orderupdate')
      
      print(order_data)

  def open_callback():
      global feed_opened
      feed_opened = True


  def close_callback():
      global feed_opened
      feed_opened = False

  def setupWebSocket() :

    api.close_websocket()
    global fee_opened
    
    api.start_websocket( order_update_callback=event_handler_order_update,
                      subscribe_callback=event_handler_feed_update, 
                      socket_open_callback= open_callback,
                        socket_close_callback = close_callback)

    time.sleep(1)
    
    while(feed_opened==False):
        # print('here only')
        pass


    return True 



In [5]:
if True : 
  if True:
    from NorenRestApiPy.NorenApi import  NorenApi
    from threading import Timer
    import pandas as pd
    import time
    import concurrent.futures

    api = None
    class Order:
        def __init__(self, buy_or_sell:str = None, product_type:str = None,
                    exchange: str = None, tradingsymbol:str =None, 
                    price_type: str = None, quantity: int = None, 
                    price: float = None,trigger_price:float = None, discloseqty: int = 0,
                    retention:str = 'DAY', remarks: str = "tag",
                    order_id:str = None):
            self.buy_or_sell=buy_or_sell
            self.product_type=product_type
            self.exchange=exchange
            self.tradingsymbol=tradingsymbol
            self.quantity=quantity
            self.discloseqty=discloseqty
            self.price_type=price_type
            self.price=price
            self.trigger_price=trigger_price
            self.retention=retention
            self.remarks=remarks
            self.order_id=None


        #print(ret)

        


    def get_time(time_string):
        data = time.strptime(time_string,'%d-%m-%Y %H:%M:%S')

        return time.mktime(data)


    class ShoonyaApiPy(NorenApi):
        def __init__(self):
            NorenApi.__init__(self, host='https://api.shoonya.com/NorenWClientTP/', websocket='wss://api.shoonya.com/NorenWSTP/', eodhost='https://shoonya.finvasia.com/chartApi/getdata/')
            global api
            api = self

        def place_basket(self, orders):

            resp_err = 0
            resp_ok  = 0
            result   = []
            with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:

                future_to_url = {executor.submit(self.place_order, order): order for order in  orders}
                for future in concurrent.futures.as_completed(future_to_url):
                    url = future_to_url[future]
                try:
                    result.append(future.result())
                except Exception as exc:
                    print(exc)
                    resp_err = resp_err + 1
                else:
                    resp_ok = resp_ok + 1

            return result
                    
        def placeOrder(self,order: Order):
            ret = NorenApi.place_order(self, buy_or_sell=order.buy_or_sell, product_type=order.product_type,
                                exchange=order.exchange, tradingsymbol=order.tradingsymbol, 
                                quantity=order.quantity, discloseqty=order.discloseqty, price_type=order.price_type, 
                                price=order.price, trigger_price=order.trigger_price,
                                retention=order.retention, remarks=order.remarks)
            #print(ret)

            return ret





  def do_login():
    #enable dbug to see request and responses
    logging.basicConfig(level=logging.DEBUG)

    #start of our program
    api = ShoonyaApiPy()

    #credentials
    user        = 'FA13620'
    pwd       =  '2012Ak@sh'
    token =     "52R5K462BA4H5TJ7SS746S5534364527"
    factor2     =  pyotp.TOTP(token).now()
    vc          = 'FA13620_U'
    app_key     = '9088c808ac3522b35705a4acd8a46d03'
    imei        = 'abc1234'




    ret = api.login(userid=user, password=pwd, twoFA=factor2, vendor_code=vc, api_secret=app_key, imei=imei)


    print(factor2)

    print(ret)



  def get_token_df (opt_type= 'CE', expiry = '08-DEC-2022'):

    # read
    global selected_instrument
        
    token_df = pd.read_csv('https://shoonya.finvasia.com/NFO_symbols.txt.zip')
    
    
    filtered_token  = token_df [token_df['Symbol'] == selected_instrument]  
    filtered_token  = filtered_token [filtered_token['Expiry'] == expiry]
    filtered_token  = filtered_token [filtered_token['OptionType'] == opt_type]
    print(filtered_token)

    filtered_token.sort_values(by =  'StrikePrice', inplace = True)


    return filtered_token












  def make_ready_to_subscribe( token_df  ):

    print(token_df)

    token_list = token_df['Token'].tolist()

    
    # token_list = token_df[80:len(token_df)]['Token'].tolist()

    ready_to_subscribe = []
    for i in token_list:

      ready_to_subscribe.append( 'NFO|'+str(i) )


    print(ready_to_subscribe)


    return ready_to_subscribe







  def select_opt_by_premium (premium  = 100):
    # returns token and price of the selected premium 
    # for that it looks into ce_lp_data which is updated by the websocket 

    diff  = 1000000
    nearest_price = 1000000
    nearest_token = 9000000000

    for x, y  in ce_lp_data.items():


      if abs(float(y) - premium) < diff :

        diff = abs(float(y) - premium)
        nearest_token = x
        nearest_price = float(y)


    
      # print(x , y )

    print('nearest token , nearest_price, and difference is ')
    print(nearest_token,nearest_price , diff)


    return nearest_token , nearest_price
















In [6]:

def select_opt_by_premium_type (premium , opt_type  ):
  # returns token and price of the selected premium 
  # for that it looks into ce_lp_data which is updated by the websocket 

  diff  = 1000000
  nearest_price = 1000000
  nearest_token = 9000000000

  global ce_lp_data
  global pe_lp_data
  
  if opt_type == 'CE':

    for x, y  in ce_lp_data.items():

      if abs(float(y) - premium) < diff :

        diff = abs(float(y) - premium)
        nearest_token = x
        nearest_price = float(y)
        # print(x , y )

    print('nearest token , nearest_price, and difference is ')
    print(nearest_token,nearest_price , diff)

    return nearest_token , nearest_price




  if opt_type == 'PE' :
        
    for x, y  in pe_lp_data.items():

        if abs(float(y) - premium) < diff :

          diff = abs(float(y) - premium)
          nearest_token = x
          nearest_price = float(y)

          # print(x , y )

    print('nearest token , nearest_price, and difference is ')
    print(nearest_token,nearest_price , diff)


    return nearest_token , nearest_price

  
  return 0 , 0


def exit_position( exit_token  ):

  exit_tradingsymbol = token_to_tradingsymbol ( int(exit_token)  )

  

  ret = api.place_order(buy_or_sell='B',
                        product_type='I',
                        exchange='NFO',
                        tradingsymbol= exit_tradingsymbol,
                        quantity=25,
                        discloseqty=0, 
                        price_type='MKT', 
                        # price=200.00, 
                        # trigger_price=199.50,
                        retention='DAY',
                        remarks='mkt_order')
  




  # exit position with exit_tradingsymbol
  pass




def make_position( new_premium ,  new_type ):

  new_token, new_price  = select_opt_by_premium_type ( premium =  new_premium ,opt_type  =  new_type)



  print(' in make position ,   new token and new_price', new_token, new_price)

  new_tradingsymbol = token_to_tradingsymbol ( int(new_token) )
  print('########################### placing order of ', new_tradingsymbol )

  time.sleep(5)



  ret = api.place_order(buy_or_sell='S',
                        product_type='I',
                        exchange='NFO',
                        tradingsymbol= new_tradingsymbol,
                        quantity=25,
                        discloseqty=0, 
                        price_type='MKT', 
                        # price=200.00, 
                        # trigger_price=199.50,
                        retention='DAY',
                        remarks='paper_trading')
  
  print(ret)


  # place sell order with new_tradingsymbol ####################################

  
  if True :     # if order place successfully

    
    global sold_ltp 
    global sold_ce
    global sold_pe

    if new_type == 'CE': 
      del sold_ltp [ sold_ce]
      sold_ltp [new_token]  =   0
      sold_ce = new_token


    
    if new_type == 'PE': 
      del sold_ltp [ sold_pe]
      sold_ltp[ new_token] = 0
      sold_pe = new_token



In [7]:
def token_to_tradingsymbol ( token ):



  print(' token_to_tradingsymbol:  token passed here is ' , token  )

  global global_token_df

  try : 
    trading_symbol = global_token_df [ global_token_df [ 'Token']  == token ]

    trading_symbol = trading_symbol ['TradingSymbol'].iloc[0]
    print(trading_symbol)

    return trading_symbol

  except IndexError as e :

    print('invalid token  ')
    return 0
  
  


token_to_tradingsymbol( int( '43308') )


 token_to_tradingsymbol:  token passed here is  43308
invalid token  


0

In [8]:
#@title
def exit_position( exit_token  ):

  exit_tradingsymbol = token_to_tradingsymbol ( int(exit_token)  )

  print('inside exit position , now trying to exit position ')

  

  ret = api.place_order(buy_or_sell='B',
                        product_type='I',
                        exchange='NFO',
                        tradingsymbol= exit_tradingsymbol,
                        quantity=25,
                        discloseqty=0, 
                        price_type='MKT', 
                        # price=200.00, 
                        # trigger_price=199.50,
                        retention='DAY',
                        remarks='mkt_order')
  
  print(ret )
  




  # exit position with exit_tradingsymbol
  pass




def make_position( new_premium ,  new_type ):

  new_token, new_price  = select_opt_by_premium_type ( premium =  new_premium ,opt_type  =  new_type)



  print(' in make position ,   new token and new_price', new_token, new_price)

  new_tradingsymbol = token_to_tradingsymbol ( int(new_token) )
  print('111111111111111')
  print(new_tradingsymbol)
  print(14512145512451245845)

  time.sleep(5)



  ret = api.place_order(buy_or_sell='S',
                        product_type='I',
                        exchange='NFO',
                        tradingsymbol= new_tradingsymbol,
                        quantity=25,
                        discloseqty=0, 
                        price_type='MKT', 
                        # price=200.00, 
                        # trigger_price=199.50,
                        retention='DAY',
                        remarks='mkt_order')
  
  print(ret)


  # place sell order with new_tradingsymbol ####################################

  
  if True :     # if order place successfully

    
    global sold_ltp 
    global sold_ce
    global sold_pe

    if new_type == 'CE': 
      del sold_ltp [ sold_ce]
      sold_ltp [new_token]  =   0
      sold_ce = new_token



      




    
    if new_type == 'PE': 
      del sold_ltp [ sold_pe]
      sold_ltp[ new_token] = 0
      sold_pe = new_token




In [9]:

do_login()

global token_df
ready_to_subscribe = make_ready_to_subscribe(token_df)


# it starts websocket 


global feed_opened 
if feed_opened == True :

  api.close_websocket()
  time.sleep(1)
  setupWebSocket()
else:
  setupWebSocket()
  feed_opened = True


time.sleep(5)




561746
{'request_time': '09:42:23 09-12-2022', 'actid': 'FA13620', 'uname': 'AKASH RAJESHKUMAR PATEL', 'prarr': [{'prd': 'C', 's_prdt_ali': 'CNC', 'exch': ['NSE', 'BSE', 'NIPO']}, {'prd': 'M', 's_prdt_ali': 'NRML', 'exch': ['NFO', 'CDS', 'BCD', 'MCX']}, {'prd': 'I', 's_prdt_ali': 'MIS', 'exch': ['NSE', 'BSE', 'NFO', 'CDS', 'BCD', 'MCX']}, {'prd': 'H', 's_prdt_ali': 'CO', 'exch': ['NSE', 'NFO', 'CDS', 'MCX', 'BSE']}, {'prd': 'B', 's_prdt_ali': 'BO', 'exch': ['NSE', 'NFO', 'CDS', 'MCX', 'BSE']}], 'stat': 'Ok', 'susertoken': '43f8196cf8f7764404b3abd0c9bd8931723bf637522d1637290491cda2e29e07', 'email': 'AKSPMONEY1@GMAIL.COM', 'uid': 'FA13620', 'brnchid': 'HO', 'orarr': ['LMT', 'MKT', 'SL-LMT', 'SL-MKT'], 'exarr': ['NSE', 'NFO', 'CDS', 'MCX', 'BSE', 'BCD', 'NIPO'], 'values': ['1'], 'mws': {'1': []}, 'brkname': 'FINV', 'lastaccesstime': '1670559143'}
      Exchange  Token  LotSize     Symbol           TradingSymbol  \
81038      NFO  45195       25  BANKNIFTY  BANKNIFTY15DEC22C31500   
81037 

In [10]:


# api.unsubscribe(ready_to_subscribe)

api.subscribe(ready_to_subscribe, feed_type='t')



In [11]:
# api.close_websocket()
# feed_opened = False

In [16]:
if sold_ce == 1 and sold_pe  == 2  or True: 

   
  make_position ( new_premium =  premium  ,  new_type = 'CE' )
  make_position ( new_premium =  premium  ,  new_type = 'PE' )


  pass


while True:

  if float(sold_ltp [sold_ce])  > sl  or  float (sold_ltp[sold_pe]  ) > sl: 
    

    if sl > max_sl : 
      pass
      # exit all position and stop trading

      exit_position( sold_ce)
      exit_position (sold_pe) 

      sys.exit(0)




    if float(sold_ltp[sold_ce]) > sl : 
      pass

      exit_position(  sold_pe   )

      make_position ( new_premium =  premium *  new_position_multiplier  ,  new_type = 'PE' )

      # global premium ,sl
      premium = new_position_multiplier * premium 
      sl = premium * sl_multiplier

      
        # it will update sold_ce , sl , premium , sold_ltp 

    if float(sold_ltp[sold_pe]) > sl : 
      pass
      
      exit_position(  sold_ce   )
      
      premium 
      make_position ( new_premium =  premium  * new_position_multiplier  ,  new_type = 'CE' )
      
      # global premium ,sl
      premium = new_position_multiplier * premium 
      sl = premium * sl_multiplier




nearest token , nearest_price, and difference is 
45780 25.2 0.1999999999999993
 in make position ,   new token and new_price 45780 25.2
 token_to_tradingsymbol:  token passed here is  45780
BANKNIFTY15DEC22C44600
111111111111111
BANKNIFTY15DEC22C44600
14512145512451245845
{'request_time': '09:44:10 09-12-2022', 'stat': 'Ok', 'norenordno': '22120900081134'}
nearest token , nearest_price, and difference is 
45654 25.35 0.3500000000000014
 in make position ,   new token and new_price 45654 25.35
 token_to_tradingsymbol:  token passed here is  45654
BANKNIFTY15DEC22P42500
111111111111111
BANKNIFTY15DEC22P42500
14512145512451245845
orderupdate
{'t': 'om', 'norenordno': '22120900081134', 'uid': 'FA13620', 'actid': 'FA13620', 'exch': 'NFO', 'tsym': 'BANKNIFTY15DEC22C44600', 'trantype': 'S', 'qty': '25', 'prc': '0.00', 'pcode': 'I', 'remarks': 'mkt_order', 'status': 'PENDING', 'reporttype': 'NewAck', 'prctyp': 'MKT', 'ret': 'DAY', 'exchordid': '', 'dscqty': '0'}
orderupdate
{'t': 'om', 'noren

KeyboardInterrupt: ignored

In [ ]:
# import datetime as dt 
# import pytz


# dt1 = dt.datetime.now()
# print(dt1 )


# dt2 = dt.datetime.now( pytz.timezone( "Asia/Calcutta" ) ).time()
# print(dt2)




# dt3 = dt.datetime.local_time()


# print(dt3)

In [ ]:
# pprint.pprint(api.get_order_book())




# order_book  = api.get_order_book()

# if order_book[0]['status']  == 'REJECTED' :
#   print( ' rejected order is this ')

In [14]:
print( ce_lp_data)

print(pe_lp_data)


print( lp_data)


{'45195': '12136.85', '45197': '11637.45', '45201': '11138.10', '45203': '10638.75', '45210': '10139.35', '45213': '9640.00', '45216': '9140.65', '45219': '8641.30', '45231': '8141.90', '45233': '7642.55', '45235': '7231.45', '45237': '6586.40', '45239': '6144.45', '45241': '6044.60', '45243': '5944.70', '45247': '5844.85', '45266': '5745.00', '45276': '5738.10', '45279': '5545.25', '45281': '5445.40', '45308': '5345.55', '45314': '5245.70', '45326': '5025.35', '45330': '5046.00', '45337': '4946.15', '45339': '4846.35', '45341': '4746.55', '45346': '4600.50', '45360': '4547.05', '45362': '4447.35', '45364': '4347.65', '45379': '4248.05', '45398': '4228.60', '45400': '3991.65', '45408': '3949.55', '45411': '3850.25', '45435': '3751.05', '45437': '3721.70', '45448': '3426.60', '45452': '3454.45', '45454': '2525.00', '45460': '3257.80', '45466': '3229.20', '45471': '3062.35', '45474': '2860.00', '45495': '2610.00', '45503': '2772.25', '45539': '2704.90', '45544': '2123.60', '45546': '2536

In [ ]:


# print(sold_ltp [sold_ce])


# print(type(float(sold_ltp [sold_ce])))





Variable exist.
